In [2]:
import os
import cv2
import numpy as np
class Sequence_t(object):

    # def __init__(self, name: str, dataset='/data1/gaoshang/vot_rgbd2020/sequences/'):
    def __init__(self, name: str, dataset='/ssd-sata1/gs/uav-test/'):
        self._name = name
        self._dataset = dataset
        self._path = self._dataset + self._name
        self._numframe = self.num_frame

    @property
    def num_frame(self):
        try:
            seq_list = os.listdir(os.path.join(self._path, 'color'))
        except:
            print('error')
        return len(seq_list)
    @property
    def framelist(self):
        try:
            seq_list = os.listdir(os.path.join(self._path, 'color'))
        except:
            print('error')
        return seq_list
    @property
    def name(self) -> str:
        return self._name

    @property
    def identifier(self) -> str:
        return self._name

    @property
    def dataset(self):
        return self._dataset

    @property
    def gt(self):
        gtfile = os.path.join(self._path, 'groundtruth.txt')
        with open(gtfile, 'r') as f:
            value = np.loadtxt(f, delimiter=',')
        
        return value 



In [3]:
import os
import numpy as np
import logging


class Tracking(object):
    def __init__(self, name: str, path='/ssd-nvme1/gs/HCAT/pysot_toolkit/results/'):
        self._name = name
        self._path = os.path.join(path, self._name)
        self._seqlist = self.re_list()
        self._numseq = len(self._seqlist)
        self._lackseq = [] 
        self._votpath = os.path.join(self._path,'more-data-search4-test-depthflyting-early-noM-train-95pth')

    @property
    def name(self) -> str:
        return self._name
    @property
    def prre(self):
        return self._prre
    @property
    def _votseqlist(self):
        return self.vot_list()
        
    def lack(self, seq):
        self._lackseq.append(seq)
    def re_list(self):
        seq_list = []
        all_list = os.listdir(self._path)
        for name in all_list:
            if '_001.txt' in name:
                seq = name.split('_001.txt')[0]
                seq_list.append(seq)
        return seq_list
    def vot_list(self):

        all_list = os.listdir(self._votpath)

        return all_list

    def vot_prebox_conf(self, sequence):
        
        # boxtxt = os.path.join(self._votpath, sequence,'{}_001.txt'.format(sequence))
        boxtxt = os.path.join(self._votpath,'{}.txt'.format(sequence))
        if os.path.exists(boxtxt):
            pass
        else:
            boxtxt = os.path.join(self._path, '{}_001.txt'.format(sequence))
        try:
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        except:
            logging.debug('use \ t in {}'.format(self._name))
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter='\t', skiprows=1)
        
        return pre_value


    
     


def read_frame(framelist):
    frame_list = []
    for id, frameid in enumerate(framelist):

        try:
            frameid = frameid.split("_")[2]
            frameid = frameid.split(".")[0]
            frameid = int(frameid)
        except:
            frameid = int(frameid)
        frame_list.append(frameid)
    return frame_list            

def visual_rgbcolormap(savepath,sequence:Sequence_t, trackers_result,trackerlist, gtbox, colormap_alpha):
    trackerslist = trackerlist
    gt_bbox = gtbox
    trackers_bbox = trackers_result

    colorpath = os.path.join(sequence._path, 'color')
    depthpath = os.path.join(sequence._path, 'depth')
    colorimageindex = os.listdir(colorpath)
    colorimageindex.sort()
    depthimageindex = os.listdir(depthpath)
    depthimageindex.sort()
    colorimagepath = [os.path.join(colorpath, filename) for i, filename in enumerate(colorimageindex)]
    depthimagepath = [os.path.join(depthpath, filename) for i, filename in enumerate(depthimageindex)]

    # line_color
    red = (0, 0, 255)
    green = (0, 255, 0)
    grey = (128, 128, 128)
    yellow = (10, 215, 255)
    blue = (255, 0, 0)
    puple = (128, 0, 128)
    orange = (0, 69, 255)
    line_color = [yellow,red,grey, blue, puple ,orange, green]

    line_width = [2,2,2,2,2,2,2]
    save_path_r = os.path.join(savepath, sequence.name, 'color')
    save_path_d = os.path.join(savepath, sequence.name, 'colormap')
    save_path_depth = os.path.join(savepath, sequence.name, 'depth')
    if not os.path.exists(save_path_r) or not os.path.exists(save_path_d):
        os.makedirs(save_path_r)
        os.makedirs(save_path_d)
        # os.makedirs(save_path_depth)
    for i in range(1,sequence.num_frame):
        savename_r = os.path.join(save_path_r, '{}'.format(os.path.basename(colorimageindex[i])))
        if os.path.exists(savename_r):
            continue
        savename_d = os.path.join(save_path_d, '{}'.format(os.path.basename(depthimageindex[i])))
        savename_depth = os.path.join(save_path_depth, '{}'.format(os.path.basename(depthimageindex[i])))
        savename_all = [savename_r, savename_d, savename_depth]
        originimage_r = cv2.imread(colorimagepath[i])
        originimage_depth = cv2.imread(depthimagepath[i], -1)
        originimage_d = cv2.imread(depthimagepath[i], -1)
        originimage_d = cv2.normalize(originimage_d, None, 0, 255, cv2.NORM_MINMAX)
        originimage_d = cv2.applyColorMap(cv2.convertScaleAbs(originimage_d, alpha=colormap_alpha), cv2.COLORMAP_JET)
        # gt bbox plot
        # for ii, originimage in enumerate([originimage_r, originimage_d, originimage_depth]):
        for ii, originimage in enumerate([originimage_r]):
            savename = savename_all[ii] 
            
            try:
                cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), (128, 0, 128), thickness=line_width[0])
                cv2.line(originimage, (int(gt_bbox[i][0]),int(gt_bbox[i][1])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), (128, 0, 128), thickness=line_width[0])
                cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]), int(gt_bbox[i][1]+gt_bbox[i][3])), (128, 0, 128), thickness=line_width[0])
                cv2.line(originimage, (int(gt_bbox[i][0] + gt_bbox[i][2]),int(gt_bbox[i][1]+gt_bbox[i][3])), (int(gt_bbox[i][0]+gt_bbox[i][2]), int(gt_bbox[i][1])), (128, 0, 128), thickness=line_width[0])
            except:
                savename = savename_all[ii].split(".")[0]+'_gtloss' + '.' + savename_all[ii].split(".")[1]
            if ii == 0:
                try: 
                    for j in range(len(trackerslist)):
                        left_x = trackers_bbox[j][i-1][0]
                        left_y = trackers_bbox[j][i-1][1]
                        width =  trackers_bbox[j][i-1][2]
                        height = trackers_bbox[j][i-1][3]
                        try:
                            cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x+width), int(left_y)), line_color[j], thickness=line_width[0])
                            cv2.line(originimage, (int(left_x),int(left_y)), (int(left_x), int(left_y+height)), line_color[j], thickness=line_width[0])
                            cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x+width), int(left_y)), line_color[j], thickness=line_width[0])
                            cv2.line(originimage, (int(left_x+width),int(left_y+height)), (int(left_x), int(left_y+height)), line_color[j], thickness=line_width[0])
                        except:
                            savename = savename_all[ii].split(".")[0]+'_loss_{}'.format(trackerslist[j]) + '.' +savename_all[ii].split(".")[1]
                            continue    
                except:
                    continue
            cv2.imwrite(savename, originimage)
            # print(savename)


In [4]:
pcolormap = [3.4,3.4,6, 1,1.1, 5.4,0.8, 1,1.4]

sequence_listQ = os.listdir('/ssd-sata1/gs/uav-test/')
sequence_listQ.remove("DepthFlying-test.json")

# sequence_listQ = ['dark01']
print(sequence_listQ)
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
# trackers_list = ['DeT', 'iiau_rgbd', 'DAL', 'TALGD',  'CSRKCF']
# line_color = [yellow,grey, blue, puple, red ,orange, green]
trackers_list =['DepthFlying'] 
all_trackers = [Tracking(tracker) for i,tracker in enumerate(trackers_list)]
all_sequence = [Sequence_t(sequence_listQ[i]) for i in range(len(sequence_listQ))]

save_pltdepth = '/ssd-nvme1/gs/HCAT/vis/more-data-search4-test-depthflyting-early-noM-train-95pth'
if not os.path.exists(save_pltdepth):
    os.makedirs(save_pltdepth)
for index, sequence in enumerate(all_sequence):
    # if not index == 3:
    #     continue 
    print('{} start'.format(sequence.name))
    gt_box = sequence.gt
    trackers_box = []

    for i, trackers in enumerate(all_trackers): 
        prebbox = trackers.vot_prebox_conf(sequence.name)
        trackers_box.append(prebbox)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
    visual_rgbcolormap(save_pltdepth,sequence,trackers_box,trackers_list,gt_box, pcolormap[0])
    print('{} finished'.format(sequence.name))

# ,'box_darkroom_noocc_10','box_darkroom_noocc_2','box_darkroom_noocc_3','box_darkroom_noocc_4','box_darkroom_noocc_5','box_darkroom_noocc_6',
# 'box_darkroom_noocc_7',
# 'box_darkroom_noocc_8',
# 'box_darkroom_noocc_9'


['DJI_0134', 'chair_present02', 'DJI_0136', 'DJI_0139', 'glove_2', 'book_1', 'dji_bed_hotel', 'dji_gym03', 'dji_cat01', 'bikeman', 'hat01', 'outtennis01', 'outbasketball01', 'dji_gym05', 'outbasketball02', 'projector', 'bike01', 'board01', 'corn02', 'bike04', 'dji_windowman_hotel', 'bike02', 'blackfoam', 'tree', 'ielts01', 'disinfect', 'corn', 'worksite', 'DJI_0140', 'hotgloves', 'glove_1', 'DJI_0125', 'DJI_0138', 'bike_2', 'DJI_0131', 'bike03', 'back_1', 'kitchen02', 'human02_corridor', 'blueman', 'outtennis02', 'book_2', 'dji_pixhawk', 'bottle_2', 'back_2', 'bottle_1', 'DJI_0132']
DJI_0134 start
DJI_0134 finished
chair_present02 start
chair_present02 finished
DJI_0136 start
DJI_0136 finished
DJI_0139 start
DJI_0139 finished
glove_2 start
glove_2 finished
book_1 start
book_1 finished
dji_bed_hotel start
dji_bed_hotel finished
dji_gym03 start
dji_gym03 finished
dji_cat01 start
dji_cat01 finished
bikeman start
bikeman finished
hat01 start
hat01 finished
outtennis01 start
outtennis01 fin

In [2]:

import cv2
import glob
def video_write(merge_list, seq, image_size):
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    video_writer = cv2.VideoWriter(os.path.join('/ssd-nvme1/gs/HCAT/vis/No-threshold-search4d15-test-depthflyting-early-noM-train-20pth/', '%s.mp4' % seq), fourcc, 30, image_size)

    color_image_sum = len(merge_list)

    for i in range(color_image_sum):
        try:
            
            video_writer.write(cv2.imread(merge_list[i]))
        except RuntimeError as e:
            print('error')


    video_writer.release()


convert_image_path = '/ssd-nvme1/gs/HCAT/vis/No-threshold-search4d15-test-depthflyting-early-noM-train-20pth/'

fps = 20

sequence_list = os.listdir(convert_image_path)
# sequence_list = ['glove_2']
for sequence_name in sequence_list:
    sequence_path = os.path.join(convert_image_path, sequence_name)
    if os.path.isfile(sequence_path):
        continue
    imgs_path = os.path.join(sequence_path, 'color')
    imgs = os.listdir(imgs_path)
    imgs.sort()
    img_list  = []
    for img in imgs:
        img_list.append(os.path.join(imgs_path, img))
    # img_list = sorted(glob.glob(sequence_path + '/' + 'color' + "/*.jpg"))
    # print(sequence_name)
    # print(img_list)
    read_img = cv2.imread(img_list[0]).shape
    size = (read_img[1], read_img[0])
    video_write(img_list, sequence_name,size)
    # img_list.sort()
    # print(len(img_list))
    # read_img = cv2.imread(img_list[0]).shape
    # size = (read_img[0], read_img[1])
    # videoWriter = cv2.VideoWriter(os.path.join('/data1/gaoshang/vot_rgbd2020/video/',sequence_name,'.mp4'),cv2.VideoWriter_fourcc('m', 'p', '4', 'v'),
    #                           fps,size)

    
    # for img in img_list:

    #     read_img = cv2.imread(img)
    #     videoWriter.write(read_img)
    # videoWriter.release()
    # print('{} finish'.format(sequence_name))



    



#videoWriter = cv2.VideoWriter('/data1/goashang/vot_rgbd2020/vislization/video',cv2.VideoWriter_force(‘X’, ‘V’, ‘I’, ‘D’), fps,size)
